In [1]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData
Base = declarative_base()

In [2]:
from sqlalchemy import Column, Integer, String, Float

In [4]:
# Create Shark Attack Classes
# ----------------------------------
class Shark_Attacks (Base):
    __tablename__ = 'shark_attacks'
    year = Column(Integer, primary_key=True)
    type = Column(String(255))
    country = Column(String(255))
    area = Column(String(255))
    activity = Column(String(255))
    fatal = Column(String(1))
    species = Column(String(255))

In [5]:
# Create Database Connection
database_path = "sharkattacks_db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [6]:
import pandas as pd
csv_path = "./filtered_shark_attacks_v2.csv"
df = pd.read_csv("./filtered_shark_attacks_v2.csv")

new_df = df.rename(columns={"Fatal (Y/N)":"Fatal"})
clean_new_df = new_df.drop(columns=['Unnamed: 7', 'Unnamed: 8'])


In [7]:
df

,Year,Type,Country,Area,Activity,Fatal,Species,Unnamed: 7,Unnamed: 8
0,2020.0,Unprovoked,USA,Maui,Stand-Up Paddle boarding,N,Tiger shark,NaN,NaN
1,2020.0,Provoked,BAHAMAS,Exumas,Floating,N,NaN,NaN,NaN
2,2020.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,N,"""A small shark""",NaN,NaN
3,2020.0,Unprovoked,NEW ZEALAND,Southland,Surfing,N,Broadnose seven gill shark?,NaN,NaN
4,2020.0,Unprovoked,USA,North Carolina,Surfing,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6457,0.0,Unprovoked,AUSTRALIA,Western Australia,Diving,Y,NaN,NaN,NaN
6458,0.0,Unprovoked,AUSTRALIA,Western Australia,Pearl diving,Y,NaN,NaN,NaN
6459,0.0,Unprovoked,USA,North Carolina,Swimming,Y,NaN,NaN,NaN
6460,0.0,Unprovoked,PANAMA,NaN,NaN,Y,NaN,NaN,NaN


In [8]:
clean_new_df

,Year,Type,Country,Area,Activity,Fatal,Species
0,2020.0,Unprovoked,USA,Maui,Stand-Up Paddle boarding,N,Tiger shark
1,2020.0,Provoked,BAHAMAS,Exumas,Floating,N,NaN
2,2020.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,N,"""A small shark"""
3,2020.0,Unprovoked,NEW ZEALAND,Southland,Surfing,N,Broadnose seven gill shark?
4,2020.0,Unprovoked,USA,North Carolina,Surfing,N,NaN
...,...,...,...,...,...,...,...
6457,0.0,Unprovoked,AUSTRALIA,Western Australia,Diving,Y,NaN
6458,0.0,Unprovoked,AUSTRALIA,Western Australia,Pearl diving,Y,NaN
6459,0.0,Unprovoked,USA,North Carolina,Swimming,Y,NaN
6460,0.0,Unprovoked,PANAMA,NaN,NaN,Y,NaN


In [9]:
clean_new_df.to_sql('shark_attacks', con=engine,  if_exists='replace')

In [10]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [11]:
# Create a Session Object to Connect to DB
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [12]:
from sqlalchemy import inspect
from sqlalchemy import Column
inspector = inspect(engine)


# #Get table information
# print(inspector.get_table_names())
# table = inspector.get_table_names()

# #Get column information
# print(inspector.get_columns('shark_attacks'))

In [13]:
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)
    shark_attacks_table = table.name
    for column in table.c:
        print(column.name)
    

shark_attacks
index
Year
Type
Country
Area
Activity
Fatal
Species 


In [20]:
from sqlalchemy import select
table = m.tables['shark_attacks']

#Select * from shark_attacks
select_statement = select([table])
res = conn.execute(select_statement)
print(res)
# for row in res:
#     print(row)

In [15]:
#Select * from shark_attacks
select_statement = select([table]).where(table.c.Country == 'USA')
res = conn.execute(select_statement) #error here
for row in res:
    print(row)

(0, 2020.0, 'Unprovoked', 'USA', 'Maui', 'Stand-Up Paddle boarding', 'N', 'Tiger shark')
(4, 2020.0, 'Unprovoked', 'USA', 'North Carolina', 'Surfing', 'N', None)
(15, 2019.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', None)
(19, 2019.0, 'Unprovoked', 'USA', 'Hawaii', 'Stand-Up Paddleboarding', 'N', "10'-12' tiger shark")
(20, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Juvenile blacktip shark')
(26, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' blacktip shark")
(31, 2019.0, 'Provoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' shark")
(34, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(36, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(37, 2019.0, 'Unprovoked', 'USA', 'Guam', 'Surfing', 'N', 'Juvenile shark')
(39, 2019.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', "6' to 7' shark")
(40, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(41, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N',

(893, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(895, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(896, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(897, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(898, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(899, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(900, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(901, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Boogie boarding', 'N', "3.5' to 4' shark")
(902, 2012.0, 'Provoked', 'USA', 'Hawaii', 'Spearfishing', 'N', "Tiger shark, 10' to 12'")
(903, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Swimming or boogie boarding', 'N', "3.5' to 4' shark")
(907, 2012.0, 'Unprovoked', 'USA', 'Alabama', 'Wading or swimming', 'N', None)
(909, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Free diving', 'N', "Lemon shark, 4' to 5'")
(912, 2012.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', None)

(1516, 2007.0, 'Invalid', 'USA', 'California', 'Swimming', None, 'Shark involvement not confirmed')
(1517, 2007.0, 'Unprovoked', 'USA', 'North Carolina', 'Wading', 'N', "5' shark")
(1519, 2007.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(1522, 2007.0, 'Provoked', 'USA', 'Florida', 'Removing hook from shark', 'N', "Lemon shark, 6' female")
(1523, 2007.0, 'Unprovoked', 'USA', 'California', 'Swimming', 'N', None)
(1524, 2007.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "4' shark")
(1525, 2007.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', 'Tiger shark')
(1526, 2007.0, 'Invalid', 'USA', 'Florida', 'Surfing', None, 'Shark involvement not confirmed')
(1527, 2007.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', "3' to 4' shark")
(1529, 2007.0, 'Unprovoked', 'USA', 'South Carolina', 'Wading', 'N', None)
(1530, 2007.0, 'Provoked', 'USA', 'Virginia', 'Reviving a sedated shark', 'N', 'A 10-year-old 94-pound pregnant blacktip reef shark')
(1534, 2007.0, 'Unprovoked', 'USA'

(1968, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Boogie boarding', 'N', "3' to 4' shark")
(1970, 2002.0, 'Unprovoked', 'USA', 'Hawaii', 'Swimming', 'N', 'A “small” shark')
(1972, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(1973, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Blacktip or spinner shark')
(1974, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "3.5' to 4.5' shark")
(1975, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "3.5' to 4' shark")
(1976, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' shark")
(1977, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(1978, 2002.0, 'Unprovoked', 'USA', 'Hawaii', 'Fishing from Surfboard', 'N', "1.2 m [4'] blacktip shark")
(1979, 2002.0, 'Provoked', 'USA', 'Florida', 'Fishing', 'N', "1.8 m [6'] blacktip shark")
(1981, 2002.0, 'Unprovoked', 'USA', 'Oregon', 'Boogie boarding or Surfing', 'N', "2.4 m [8'] shark")
(1982, 2002.0, 'Unprovoked', 'USA', 'California', 'Su

(3078, 1983.0, 'Unprovoked', 'USA', 'Virginia', 'Swimming', 'N', 'Sand shark')
(3079, 1983.0, 'Provoked', 'USA', 'Florida', 'Skindiving', 'N', 'Nurse shark')
(3080, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Diving', 'N', "6' to 8' bull shark")
(3081, 1983.0, 'Boat', 'USA', 'Rhode Island', 'Fishing', 'N', None)
(3086, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Windsurfing', 'N', "3.5' hammerhead shark")
(3088, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Floating on air mattress', 'N', None)
(3089, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(3091, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Surfing & dangling foot in water amid baitfish', 'N', None)
(3095, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Water-skiing', 'N', "6' shark")
(3096, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(3097, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(3098, 1983.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'UNKNOWN', None)
(3100, 1983.0, 'Unprovoked', 'USA', 'Flo

(4348, 1957.0, 'Provoked', 'USA', 'California', 'Spearfishing on Scuba', 'N', 'Said to involve a 1 m white shark, but thought that it was more likely a blue shark')
(4349, 1957.0, 'Unprovoked', 'USA', 'North Carolina', 'Swimming', 'Y', 'White shark')
(4354, 1957.0, 'Unprovoked', 'USA', 'Florida', '3 men & 2 boys picked up wearing life jackets and with inner tube', 'N', None)
(4355, 1957.0, 'Unprovoked', 'USA', 'California', 'Swimming', 'Y', 'White shark')
(4357, 1957.0, 'Provoked', 'USA', 'Florida', 'Standing', 'N', "4 m [13'] shark")
(4362, 1957.0, 'Unprovoked', 'USA', 'Florida', 'Floating', 'N', "1.2 m [4'] shark")
(4364, 1957.0, 'Unprovoked', 'USA', 'Florida', 'Skindiving for specimens', 'N', 'Said to involve a large mako shark')
(4367, 1957.0, 'Unprovoked', 'USA', 'Florida', 'Bathing', 'N', "1.5 m [5'] shark")
(4371, 1957.0, 'Unprovoked', 'USA', 'Florida', 'Helmet diving in Miami Seaquarium', 'N', 'Bull shark')
(4385, 1956.0, 'Unprovoked', 'USA', 'Florida', 'Surf fishing', 'N', 'a 

NameError: name 'shark_attacks' is not defined